# C++ Cling Examples for STM32F767X board

![Image](img/board.jpg)

Load [REMCU](https://remotemcu.github.io) shared libray

In [ ]:
.L libremcu

Add path with header files

In [ ]:
.I remcu_include/

Including necessary header files. The [“remcu.h”](remcu_include/remcu.h) header must be always included before any MCU header files.

In [ ]:
#include "remcu.h"

Connection to a debug server. The [**REMCU**](https://remotemcu.github.io/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/).
There is using the [remcu_connect2OpenOCD](https://remotemcu.github.io/api-v1-0#remcu_getVersion) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)

Before that, plug stm32 board with debugger to PC

run openocd utility in command line:
```
openocd -f board/stm32f7discovery.cfg
```
Result:
![Image](img/openocd_unix.png)

In [ ]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const int timeout_sec = 3; // It can not be negative

In [ ]:
remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server  etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [ ]:
//const uint16_t default_openocd_gdb_port = 3333;
//const uint16_t default_jlink_port = 2331;
//const uint16_t default_stlink_port = 61234;
//const uint16_t custom_port = ...;
//remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port /*default_jlink_port*/ /*default_stlink_port*/, timeout_sec)

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page) 
    
If connection were success... Jupyter Output:
![Image](img/success_connection_for_unix.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)):

In [ ]:
remcu_resetRemoteUnit(__HALT)

HAL Low Level header files:

In [ ]:
#include "stm32f7xx_ll_adc.h"
#include "stm32f7xx_ll_dma.h"
#include "stm32f7xx_ll_tim.h"
#include "stm32f7xx_ll_dac.h"
#include "stm32f7xx_ll_gpio.h"
#include "stm32f7xx_ll_bus.h"
#include "stm32f7xx_ll_rcc.h"

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (system_stm32f7xx.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/system_stm32f7xx.c file

In [ ]:
SystemInit()

Simple Clock configuration:

In [ ]:
 /* Enable HSI and wait for HSI ready*/
  LL_RCC_HSI_Enable();

  while(LL_RCC_HSI_IsReady() != 1)

In [ ]:
/* Select HSI as system clock */
/* Wait for HSI switched */
  LL_RCC_SetSysClkSource(LL_RCC_SYS_CLKSOURCE_HSI);

### PWM example
![Image](img/pwm_wave.png)
This example shows how to configure the TIM peripheral in PWM (Pulse Width Modulation) mode.

>The code below have been based on **TIM_PWMOutput** ([main.c](https://github.com/STMicroelectronics/STM32CubeF7/blob/master/Projects/STM32F767ZI-Nucleo/Examples_LL/TIM/TIM_PWMOutput/Src/main.c)) example of [HAL LL Library for STM32F767ZI-Nucleo](https://github.com/STMicroelectronics/STM32CubeF7/tree/master/Projects/STM32F767ZI-Nucleo/Examples_LL/TIM/TIM_PWMOutput)

Use of a timer peripheral to generate a 
PWM output signal and update the PWM duty cycle. This example is based on the 
STM32F7xx TIM LL API. The peripheral initialization uses 
LL initialization function to demonstrate LL init.

In this example TIM3 input clock (TIM3CLK) frequency is set to APB1 clock (PCLK1),
since APB1 pre-scaler is equal to 2 and it is twice PCLK1.
    TIM3CLK = 2*PCLK1
    PCLK1 = HCLK/2
    => TIM3CLK = (2/2)*HCLK = SystemCoreClock (216 Mhz)

To set the TIM3 counter clock frequency to 10 KHz, the pre-scaler (PSC) is
calculated as follows:
PSC = (TIM3CLK / TIM3 counter clock) - 1
PSC = (SystemCoreClock /10 KHz) - 1

SystemCoreClock is set to 216 MHz for STM32F7xx Devices.

Auto-reload (ARR) is calculated to get a time base period of 10 ms,
meaning a time base frequency of 100 Hz.
ARR = (TIM3 counter clock / time base frequency) - 1
ARR = (TIM3 counter clock / 100) - 1

Initially, the capture/compare register (CCR3) of the output channel is set to
half the auto-reload value meaning a initial duty cycle of 50%.
Generally speaking this duty cycle is calculated as follows:
Duty cycle = (CCR3 / ARR) * 100

The timer output channel is mapped on the pin PB.0 (connected to LED1 on board
NUCLEO-F767ZI). Thus LED1 status (on/off) mirrors the timer output
level (active v.s. inactive).

User push-button can be used to change the duty cycle from 0% up to 100% by
steps of 10%. Duty cycle is periodically measured. It can be observed through
the debugger by watching the variable uwMeasuredDutyCycle.

@Note It is recommended to enable the cache and maintain its coherence, but depending on the use case
      It is also possible to configure the MPU as "Write through", to guarantee the write access coherence.
      In that case, the MPU must be configured as Cacheable/Bufferable/Not Shareable.
      Even though the user must manage the cache coherence for read accesses.
      Please refer to the AN4838 “Managing memory protection unit (MPU) in STM32 MCUs”
      Please refer to the AN4839 “Level 1 cache on STM32F7 Series”

@par Hardware and Software environment

  - This example runs on STM32F767xx devices.
    
  - This example has been tested with NUCLEO-F767ZI board and can be
    easily tailored to any other supported device and development board.


In [ ]:
  LL_TIM_InitTypeDef    tim_initstruct;
  LL_TIM_OC_InitTypeDef tim_oc_initstruct;
  
  /*************************/
  /* GPIO AF configuration */
  /*************************/
  /* Enable the peripheral clock of GPIOs */
  LL_AHB1_GRP1_EnableClock(LL_AHB1_GRP1_PERIPH_GPIOB);
  
  /* GPIO TIM3_CH3 configuration */
  LL_GPIO_SetPinMode(GPIOB, LL_GPIO_PIN_0, LL_GPIO_MODE_ALTERNATE);
  LL_GPIO_SetPinPull(GPIOB, LL_GPIO_PIN_0, LL_GPIO_PULL_DOWN);
  LL_GPIO_SetPinSpeed(GPIOB, LL_GPIO_PIN_0, LL_GPIO_SPEED_FREQ_HIGH);
  LL_GPIO_SetAFPin_0_7(GPIOB, LL_GPIO_PIN_0, LL_GPIO_AF_2);

  /******************************/
  /* Peripheral clocks enabling */
  /******************************/
  /* Enable the timer peripheral clock */
  LL_APB1_GRP1_EnableClock(LL_APB1_GRP1_PERIPH_TIM3); 

/***************************/
  /* Time base configuration */
  /***************************/
  /* Set fields of initialization structure */
  /* - Set the pre-scaler value to have TIM3 counter clock equal to 10 kHz  */
  /* - Set the auto-reload value to have a counter frequency of 100 Hz        */
  /* TIM3CLK = SystemCoreClock / (APB prescaler & multiplier)               */
  static uint32_t TimOutClock = SystemCoreClock/2;
  
  tim_initstruct.Prescaler         = __LL_TIM_CALC_PSC(SystemCoreClock, 10000);
  tim_initstruct.CounterMode       = LL_TIM_COUNTERMODE_UP;
  tim_initstruct.Autoreload        = __LL_TIM_CALC_ARR(TimOutClock, tim_initstruct.Prescaler, 100);
  tim_initstruct.ClockDivision     = LL_TIM_CLOCKDIVISION_DIV1;
  tim_initstruct.RepetitionCounter = (uint8_t)0x00;
  
  /* Initialize TIM instance according to parameters defined in               */
  /* initialization structure.                                                */
  LL_TIM_Init(TIM3, &tim_initstruct);
  
  /* Enable TIM3_ARR register preload. Writing to or reading from the         */
  /* auto-reload register accesses the preload register. The content of the   */
  /* preload register are transferred into the shadow register at each update */
  /* event (UEV).                                                             */  
  LL_TIM_EnableARRPreload(TIM3);

/*********************************/
  /* Output waveform configuration */
  /*********************************/
  /* Set fields of initialization structure */
  /*  - Set compare value to half of the counter period (50% duty cycle ) */
  tim_oc_initstruct.OCMode       = LL_TIM_OCMODE_PWM1;
  tim_oc_initstruct.OCState      = LL_TIM_OCSTATE_DISABLE;
  tim_oc_initstruct.OCNState     = LL_TIM_OCSTATE_DISABLE;
  tim_oc_initstruct.CompareValue = ( (LL_TIM_GetAutoReload(TIM3) + 1 ) / 2);
  tim_oc_initstruct.OCPolarity   = LL_TIM_OCPOLARITY_HIGH;
  tim_oc_initstruct.OCNPolarity  = LL_TIM_OCPOLARITY_HIGH;
  tim_oc_initstruct.OCIdleState  = LL_TIM_OCIDLESTATE_LOW;
  tim_oc_initstruct.OCNIdleState = LL_TIM_OCIDLESTATE_LOW;
  
  /* Initialize TIM instance according to parameters defined in               */
  /* initialization structure.                                                */
  LL_TIM_OC_Init(TIM3, LL_TIM_CHANNEL_CH3, &tim_oc_initstruct);
  
  /* Enable TIM3_CCR3 register preload. Read/Write operations access the      */
  /* preload register. TIM3_CCR3 preload value is loaded in the active        */
  /* at each update event.                                                    */
  LL_TIM_OC_EnablePreload(TIM3, LL_TIM_CHANNEL_CH3);
  
  /**************************/
  /* TIM3 interrupts set-up */
  /**************************/
  /* Enable the capture/compare interrupt for channel 1*/
  LL_TIM_EnableIT_CC3(TIM3);
  
  /**********************************/
  /* Start output signal generation */
  /**********************************/
  /* Enable output channel 3 */
  LL_TIM_CC_EnableChannel(TIM3, LL_TIM_CHANNEL_CH3);
  
  /* Enable counter */
  LL_TIM_EnableCounter(TIM3);
  
  /* Force update generation */
  LL_TIM_GenerateEvent_UPDATE(TIM3);

  const uint32_t Period = LL_TIM_GetAutoReload(TIM3) + 1;

In [ ]:
#include "xwidgets/xslider.hpp"
xw::slider<int> sliderBrightness_Red;

sliderBrightness_Red.min = 0;
sliderBrightness_Red.max = Period;

In [ ]:
sliderBrightness_Red.display();

XOBSERVE(sliderBrightness_Red, value, [](const auto& Brightness) {
    LL_TIM_OC_SetCompareCH3(TIM3,sliderBrightness_Red.max - Brightness.value);
});